In [191]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch.utils.data

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import os

In [2]:
tracking2018 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2018.csv")
tracking2019 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2019.csv")
tracking2020 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2020.csv")

tracking2018.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right


In [3]:
play_data = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/plays.csv")
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull() and (not returnerId.isnull())", engine='python', inplace=False)


In [21]:
data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/ball_carrier_data.pkl')

In [227]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [23]:
def split_data(dataset, train_split, seed):
    np.random.seed(seed)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)

    train_num = int(len(dataset)*train_split)
    val_num = (len(dataset) - int(len(dataset)*train_split))//2

    train_indices = indices[0:train_num]
    val_indices = indices[train_num:train_num+val_num]
    test_indices = indices[train_num+val_num:]

    #check to make sure slices correct
    assert len(dataset) == len(train_indices) + len(val_indices) + len(test_indices)

    #dataset = help.normalize(train_indices, dataset)

    train_data = dataset.iloc[train_indices,:]
    val_data = dataset.iloc[val_indices,:]
    test_data = dataset.iloc[test_indices,:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(data_df, 0.7, 2430)


In [110]:
'''
Function that adds needed features to the play df
'''

def modify_play_df(play_df):
    # shift x-axis based on target's endzone
    playDirection = play_df.loc[play_df.index[0],'playDirection']
    
    if playDirection == 'left':
        play_df.insert(len(play_df.columns),'adj_x', play_df['x'])
        play_df.insert(len(play_df.columns),'adj_y', play_df['y'] - 26.65)
        # change dir and rotation
        play_df.insert(len(play_df.columns), 'adj_o', (play_df['o'] + 270) % 360)
        play_df.insert(len(play_df.columns), 'adj_dir', (play_df['dir'] + 270) % 360)
    if playDirection == 'right':
        
        play_df.insert(len(play_df.columns),'adj_x', 120 - play_df['x'])
        play_df.insert(len(play_df.columns),'adj_y', play_df['y'] - 26.65)
        # change dir and rotation
        play_df.insert(len(play_df.columns), 'adj_o', (play_df['o'] + 90) % 360)
        play_df.insert(len(play_df.columns), 'adj_dir', (play_df['dir'] + 90) % 360)
        
    play_df['adj_o'].round(2)
    play_df['adj_dir'].round(2)
    
    play_df.insert(len(play_df.columns), 'sin_adj_o', np.sin(play_df.loc[:,'adj_o']))
    play_df.insert(len(play_df.columns), 'cos_adj_o', np.cos(play_df.loc[:,'adj_o']))
    play_df.insert(len(play_df.columns), 'sin_adj_dir', np.sin(play_df.loc[:,'adj_dir']))
    play_df.insert(len(play_df.columns), 'cos_adj_dir', np.cos(play_df.loc[:,'adj_dir']))
    
    play_df = play_df.fillna(value=0)
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_dis'] = play_df.groupby('displayName').dis.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'next_adj_x'] = play_df.groupby('displayName').adj_x.shift(-1)
    play_df.loc[:, 'next_adj_y'] = play_df.groupby('displayName').adj_y.shift(-1)
    play_df.loc[:, 'next_adj_o'] = play_df.groupby('displayName').adj_o.shift(-1)
    play_df.loc[:, 'next_adj_dir'] = play_df.groupby('displayName').adj_dir.shift(-1)
    play_df.loc[:, 'next_sin_adj_o'] = play_df.groupby('displayName').sin_adj_o.shift(-1)
    play_df.loc[:, 'next_cos_adj_o'] = play_df.groupby('displayName').cos_adj_o.shift(-1)
    play_df.loc[:, 'next_sin_adj_dir'] = play_df.groupby('displayName').sin_adj_dir.shift(-1)
    play_df.loc[:, 'next_cos_adj_dir'] = play_df.groupby('displayName').cos_adj_dir.shift(-1)
    
    # remove NAs created by shifting; i.e. remove plays where there is no next play
    play_df.dropna(axis=0, how='any', subset=['x'], inplace=True)
   
    return play_df

In [63]:
def get_distances(x, player_pos):
    dist = np.linalg.norm(player_pos.values - x.values)
    
    try:
        dist = round(dist, 3)
    except:
        dist = np.around(dist, 3)
    
    return dist

def min_teammate_distances(player_of_interest_row, time_play_df):
    team = player_of_interest_row['team_index']
    nflid = player_of_interest_row['nflId']
    new_df = time_play_df.query("team_index == @team and displayName != 'football' and nflId != @nflid")
    dists = new_df.loc[:,['adj_x','adj_y']].apply(get_distances, axis=1,args=(player_of_interest_row[['adj_x','adj_y']],))
    min_teammate_dist = min(dists)
    return min_teammate_dist

def min_opponent_distances(player_of_interest_row, time_play_df):
    team = player_of_interest_row['team_index']
    nflid = player_of_interest_row['nflId']
    new_df = time_play_df.query("team_index != @team and displayName != 'football' and nflId != @nflid")
    dists = new_df.loc[:,['adj_x','adj_y']].apply(get_distances, axis=1,args=(player_of_interest_row[['adj_x','adj_y']],))
    min_teammate_dist = min(dists)
    return min_teammate_dist

In [64]:


def store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, playerId):
    # now we can define the values for each row
    rowDict = {}
       
    ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]

    other_team_name = 'home'
    if ball_carrier_team == 'home':
        other_team_name = 'away'

    # get distance from ball carrier
    ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    time_play_df.insert(len(time_play_df.columns), 'dist_from_ball_carrier', time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['x','y']],)))

    time_play_df.insert(len(time_play_df.columns), 'ball_carrier_bool', time_play_df.nflId != returnerId)
    time_play_df.insert(len(time_play_df.columns), 'player_bool', time_play_df.nflId != playerId)
    time_play_df.insert(len(time_play_df.columns), 'football_bool', time_play_df.displayName != 'football')
    time_play_df.insert(len(time_play_df.columns), 'team_index', time_play_df.team == ball_carrier_team)

    time_play_df.sort_values(by=["player_bool", "football_bool", "team_index", "dist_from_ball_carrier"], inplace=True, ignore_index=True)

    football_index = time_play_df.index[time_play_df.team == 'football']
    no_football_df = time_play_df.query("displayName != 'football'")
    
    time_play_df.insert(len(time_play_df.columns), 'min_teammate_dist', time_play_df.apply(min_teammate_distances, axis=1, args=(no_football_df,)))
    time_play_df.insert(len(time_play_df.columns), 'min_opponent_dist', time_play_df.apply(min_teammate_distances, axis=1, args=(no_football_df,)))
    
    states = time_play_df.loc[:,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']].to_numpy()
    football_pos = time_play_df.loc[time_play_df.team == 'football',['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']].to_numpy()
    
    rowDict['season'] = season
    rowDict['gameId'] = gameId
    rowDict['playId'] = playId

    rowDict["timeIndex"] = time_index
    rowDict["time"] = time_string
    rowDict["playerId"] = playerId
    rowDict["football_pos"] = football_pos
    # add current state
    rowDict['state'] = states
    # add future states
    next_state = time_play_df.loc[:,['next_x','next_y', 'next_dis', 'next_s', 'next_a', 'next_o', 'next_dir', 'next_adj_x', 'next_adj_y', 'next_adj_o', 'next_adj_dir', 'next_sin_adj_o', 'next_cos_adj_o', 'next_sin_adj_dir', 'next_cos_adj_dir']].to_numpy()
    rowDict['next_state'] = next_state
    # add reward - defined as ball carrier's increase in x
    ball_carrier = time_play_df.query("nflId == @returnerId")
    rowDict['reward'] = round(ball_carrier['adj_x'] - ball_carrier['next_x'],3).values

    action = states[:,:-3]- next_state
    rowDict['action'] = action.round(2)
    
    return rowDict




In [65]:
'''
Function that returns the model that outputs the entire next state
'''

def return_models(device, action_size):

    model = nn.Sequential(
        nn.Linear(184,230), 
        nn.LeakyReLU(),
        nn.Linear(230,action_size*23)
    )
    
    model.to(device)
    
    model.load_state_dict(torch.load("full_dataset_model.pt"))
    
    return model

In [66]:
'''
Function that takes in a engineered feature vector from store_player_info and a model. Outputs a prediction
def get_prediction_from_true(play, model, state_size, loss_fn, device):

'''

def get_prediction_from_true(play, model, state_size, loss_fn, device):
    
    with torch.no_grad():
        
        numpy_state = play['state']
        batch_size = numpy_state.shape[0]

        true_state = torch.tensor(numpy_state[:,[3,4,7,8,11,12,13,14]].reshape(1, -1), dtype=torch.float32).to(device=device)
        true_next_state = torch.tensor(play['next_state'][:,[3,4,7,8,11,12,13,14]].reshape(1, -1), dtype=torch.float32).to(device=device)

        model.eval()  # put model to eval mode

        #compute prediction and loss
        predicted_next_state = model(true_state)
    
        loss = loss_fn(predicted_next_state, true_next_state)
    
    return predicted_next_state, true_next_state, loss


'''
Function that gets prediction from previous predicted state
'''
def get_prediction_from_pred(returnerId, playerId, time_play_df, ball_carrier_team, model, device, previous_state):
    
    no_cuda = previous_state.detach().cpu()
    df = pd.DataFrame(no_cuda)
    df.columns = ['s', 'a', 'adj_x', 'adj_y', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']

    other_team_name = 'home'
    if ball_carrier_team == 'home':
        other_team_name = 'away'

#     df['displayName'] = time_play_df.displayName
#     df['nflId'] = time_play_df.nflId
    
    df.insert(len(df.columns),'displayName', time_play_df.displayName)
    df.insert(len(df.columns),'nflId', time_play_df.nflId)
    
    

    # get distance from ball carrier
    ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, :]
    
    ######################
    #     df['dist_from_ball_carrier'] = time_play_df.loc[:,['adj_x','adj_y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['adj_x','adj_y']],))

    #     df['ball_carrier_bool'] = time_play_df.nflId != returnerId
    #     df['player_bool'] = time_play_df.nflId != playerId
    #     df['football_bool'] = time_play_df.displayName != 'football'
    #     df['team_index'] = time_play_df.team == ball_carrier_team

    #     df.sort_values(by=["player_bool", "football_bool", "team_index", "dist_from_ball_carrier"], inplace=True, ignore_index=True)

    #     no_football_df = df.query("displayName != 'football'")

    #     df['min_teammate_dist'] = df.apply(min_teammate_distances, axis=1, args=(no_football_df,))
    #     df['min_opponent_dist'] = df.apply(min_opponent_distances, axis=1, args=(no_football_df,))
    ######################
    
    df.insert(len(df.columns),'dist_from_ball_carrier', time_play_df.loc[:,['adj_x','adj_y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['adj_x','adj_y']],)))
    df.insert(len(df.columns),'ball_carrier_bool', time_play_df.nflId != returnerId)
    df.insert(len(df.columns),'player_bool', time_play_df.nflId != playerId)
    df.insert(len(df.columns),'football_bool', time_play_df.displayName != 'football')
    df.insert(len(df.columns),'team_index', time_play_df.team == ball_carrier_team)

    df.sort_values(by=["player_bool", "football_bool", "team_index", "dist_from_ball_carrier"], inplace=True, ignore_index=True)

    no_football_df = df.query("displayName != 'football'")

    df.insert(len(df.columns), 'min_teammate_dist', df.apply(min_teammate_distances, axis=1, args=(no_football_df,)))
    df.insert(len(df.columns), 'min_opponent_dist', df.apply(min_opponent_distances, axis=1, args=(no_football_df,)))
    
    arr = np.array(df.iloc[:, 0:8], dtype=np.dtype(float)).reshape(1,-1)

    tensor_input = torch.tensor(arr, dtype=torch.float32).to(device)
    
    model.eval()
    predicted_action = model(tensor_input)

    return predicted_action

In [67]:
'''
Given a play from the play csv (not the tracking csv), returns a torch tensor of the FIRST state (relative to when
the ball carrier receives the ball). Is a (23,8) tensor with columns 
['s', 'a', 'adj_x', 'adj_y', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']
'''

def get_tensor_state(play, device):
    
    gameId = play.loc['gameId']
    playId = play.loc['playId']
    returnerId = float(play.loc['returnerId'])
    season = str(gameId)[0:4]

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        return None

    playDirection = play_df.loc[play_df.index[0],'playDirection']
    # modifies in place
    play_df = modify_play_df(play_df)
    
    events = list(np.unique(play_df.event))
    
    if 'kick_received' in events or 'punt_received' in events:
        received_time = play_df.query("event == 'kick_received' or event == 'punt_received'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        return None
    if 'tackle' in events or 'out_of_bounds' in events or 'touchdown' in events or 'fumbled' in events:
        tackled_time = play_df.query("event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        return None
    
    runner_times = play_df.query("time >= @received_time and time <= @tackled_time")

    # iterate through the times
    all_times = list(np.unique(runner_times.time))
    time_string = all_times[0]
    
    time_play_df = runner_times.query("time == @time_string", inplace=False).reset_index()
    
    df_subset = time_play_df.loc[:, ['s', 'a', 'adj_x', 'adj_y', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']]
    arr = np.array(df_subset, dtype=np.dtype(float) )
    tensor_input = torch.tensor(arr, dtype=torch.float32).to(device)
    
    return tensor_input

In [68]:
'''
Home=1
Away=0
Football=2
'''

def get_home_away(play, device):

    gameId = play.loc['gameId']
    playId = play.loc['playId']
    returnerId = float(play.loc['returnerId'])
    playerId = returnerId
    season = str(gameId)[0:4]

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        return None

    playDirection = play_df.loc[play_df.index[0],'playDirection']
    # modifies in place
    play_df = modify_play_df(play_df)

    # inspect each time
    # received_time = np.unique(play_df.query("event == 'kick_received'").time)[0]
    # tackled_time = np.unique(play_df.query("event == 'tackle'").time)[0]
    
    events = list(np.unique(play_df.event))
    
    if 'kick_received' in events or 'punt_received' in events:
        received_time = play_df.query("event == 'kick_received' or event == 'punt_received'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        return None
    if 'tackle' in events or 'out_of_bounds' in events or 'touchdown' in events or 'fumbled' in events:
        tackled_time = play_df.query("event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        return None

    runner_times = play_df.query("time >= @received_time and time <= @tackled_time")

    # iterate through the times
    all_times = list(np.unique(runner_times.time))
    time_string = all_times[0]

    time_play_df = runner_times.query("time == @time_string", inplace=False).reset_index()
    
    ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]

    other_team_name = 'home'
    if ball_carrier_team == 'home':
        other_team_name = 'away'

    # get distance from ball carrier
    ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    time_play_df.insert(len(time_play_df.columns), 'dist_from_ball_carrier', time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['x','y']],)))

    time_play_df.insert(len(time_play_df.columns), 'ball_carrier_bool', time_play_df.nflId != returnerId)
    time_play_df.insert(len(time_play_df.columns), 'player_bool', time_play_df.nflId != playerId)
    time_play_df.insert(len(time_play_df.columns), 'football_bool', time_play_df.displayName != 'football')
    time_play_df.insert(len(time_play_df.columns), 'team_index', time_play_df.team == ball_carrier_team)

    time_play_df.sort_values(by=["player_bool", "football_bool", "team_index", "dist_from_ball_carrier"], inplace=True, ignore_index=True)

    team_list = [1 if x == 'home' else 0 for x in time_play_df.team]
    home_away_arr = np.array(team_list)
    
    football_index = time_play_df.index[time_play_df.team == 'football']
    
    home_away_arr[football_index] = 2
    
    return home_away_arr

In [185]:
'''
Given a single play from Returned_Plays csv, predict the play
'''

def calc_preds(play):

    state_size = len(test_data.iloc[0,8][0])
    NUM_PLAYERS = 23     # includes football
    ACTION_SIZE = 8

    #play = returned_plays.iloc[2,:]
    gameId = play.loc['gameId']
    playId = play.loc['playId']
    returnerId = float(play.loc['returnerId'])
    season = str(gameId)[0:4]

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        return None, None, None, None

    playDirection = play_df.loc[play_df.index[0],'playDirection']
    # modifies in place
    play_df = modify_play_df(play_df)

    events = list(np.unique(play_df.event))
    
    if 'kick_received' in events or 'punt_received' in events:
        received_time = play_df.query("event == 'kick_received' or event == 'punt_received'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        return None, None, None, None
    if 'tackle' in events or 'out_of_bounds' in events or 'touchdown' in events or 'fumbled' in events:
        tackled_time = play_df.query("event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        return None, None, None, None
    
    runner_times = play_df.query("time >= @received_time and time <= @tackled_time")

    full_model = return_models(device, ACTION_SIZE)
    mse_loss_fn = nn.MSELoss()

    # iterate through the times
    all_times = list(np.unique(runner_times.time))

    states_from_true = torch.zeros((len(all_times), NUM_PLAYERS, ACTION_SIZE)).to(device)
    states_from_predicted = torch.zeros((len(all_times), NUM_PLAYERS, ACTION_SIZE)).to(device)
    all_true_states = torch.zeros((len(all_times), NUM_PLAYERS, ACTION_SIZE)).to(device)
    total_loss = 0
    

    for time_index, time_string in enumerate(all_times[:-1]):

        time_play_df_orig = runner_times.query("time == @time_string", inplace=False).reset_index()
        time_play_df = time_play_df_orig.copy(deep=True)
        # should already be sorted by nflId by construction of the dataset, but doing just to make sure
        #time_play_df.sort_values(by=["nflId"], inplace=True, ignore_index=True)

        
        ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]

        if time_index == 0:
            model_preprocessed_input = store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, returnerId)

            predicted_next_state, true_next_state, loss = get_prediction_from_true(model_preprocessed_input, full_model, state_size, mse_loss_fn, device)

            predicted_next_state = predicted_next_state.view(NUM_PLAYERS, ACTION_SIZE)
            true_next_state = true_next_state.view(NUM_PLAYERS, ACTION_SIZE)

            states_from_true[time_index] = predicted_next_state
            states_from_predicted[time_index] = predicted_next_state
            all_true_states[time_index] = true_next_state

            total_loss += loss

        else:

            # get prediction from true action
            model_preprocessed_input = store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, returnerId)
            predicted_next_state, true_next_state, loss = get_prediction_from_true(model_preprocessed_input, full_model, state_size, mse_loss_fn, device)
            # get prediction from true prediction

            predicted_next_state2 = get_prediction_from_pred(returnerId, returnerId, time_play_df, ball_carrier_team, full_model, device, states_from_predicted[time_index-1].view(NUM_PLAYERS, ACTION_SIZE))

            predicted_next_state = predicted_next_state.view(NUM_PLAYERS, ACTION_SIZE)
            predicted_next_state2 = predicted_next_state2.view(NUM_PLAYERS, ACTION_SIZE)
            true_next_state = true_next_state.view(NUM_PLAYERS, ACTION_SIZE)

            states_from_true[time_index] = predicted_next_state
            states_from_predicted[time_index] = predicted_next_state2
            all_true_states[time_index] = true_next_state

            total_loss += loss

        #if time_index % 5 == 0:
        #    print(f"Done with {time_index} out of {len(all_times)}")
    

    #print("Finished play")
    
    return states_from_true, states_from_predicted, all_true_states, total_loss

In [112]:
'''
Get play identification from test data, returned play from Returned_Plays. Pass into predict function
'''

def get_returned_play(test_index, test_data, returned_plays):
    
    test_data_entry = test_data.reset_index().iloc[test_index, ]
    
    gameId = test_data_entry.loc['gameId']
    playId = test_data_entry.loc['playId']
    
    returned_plays_entry = returned_plays.query("gameId == @gameId and playId == @playId").iloc[0,:]
    
    return returned_plays_entry

In [113]:
get_returned_play(0, test_data, returned_plays)

gameId                                                           2018112506
playId                                                                  760
playDescription           A.Rosas kicks 67 yards from NYG 35 to PHI -2. ...
quarter                                                                   1
down                                                                      0
yardsToGo                                                                 0
possessionTeam                                                          NYG
specialTeamsPlayType                                                Kickoff
specialTeamsResult                                                   Return
kickerId                                                            43937.0
returnerId                                                            45599
kickBlockerId                                                           NaN
yardlineSide                                                            NYG
yardlineNumb

In [114]:
display(test_data.head())

,season,gameId,playId,playIndex,timeIndex,time,playerId,football_pos,state,next_state,reward,action,next_action
43382,2018,2018112506,760,996,19,2018-11-25T18:30:47.700,45599.0,"[[19.48, 13.78, 0.83, 8.29, 1.26, 0.0, 0.0, 19...","[[19.46, 13.88, 0.82, 8.19, 2.16, 111.23, 111....","[[20.22, 13.57, 0.83, 8.24, 2.05, 112.08, 113....",[-0.76],"[[-0.76, 0.31, -0.01, -0.05, 0.11, -0.85, -1.3...","[[-0.76, 0.35, -0.01, -0.07, 0.01, -3.65, -1.8..."
128927,2019,2019122912,3449,2941,41,2019-12-29T20:40:12.800,40130.0,"[[46.72, 43.2, 0.45, 4.29, 3.41, 0.0, 0.0, 46....","[[46.86, 43.18, 0.52, 5.09, 2.87, 282.7, 351.4...","[[46.79, 43.67, 0.49, 4.8, 2.81, 276.68, 351.0...",[0.07],"[[0.07, -0.49, 0.03, 0.29, 0.06, 6.02, 0.34, 0...","[[0.03, -0.36, 0.13, 0.82, -0.95, -20.18, -1.3..."
34065,2018,2018111102,955,780,43,2018-11-11T18:40:31.600,44837.0,"[[87.3, 39.77, 0.06, 0.63, 1.74, 0.0, 0.0, 32....","[[86.4, 39.57, 0.26, 2.06, 3.82, 264.53, 278.8...","[[86.21, 39.6, 0.19, 1.72, 3.77, 268.35, 279.8...",[-52.61],"[[0.19, -0.03, 0.07, 0.34, 0.05, -3.82, -1.0, ...","[[0.1, -0.03, 0.08, 0.35, 0.44, 45.1, -0.72, -..."
56596,2018,2018121610,379,1287,58,2018-12-16T21:37:49.600,39975.0,"[[31.94, 38.74, 0.22, 1.38, 3.68, 0.0, 0.0, 31...","[[31.97, 39.05, 0.29, 2.98, 0.57, 76.62, 334.8...","[[31.85, 39.31, 0.29, 2.9, 0.39, 88.39, 334.67...",[0.12],"[[0.12, -0.26, 0.0, 0.08, 0.18, -11.77, 0.21, ...","[[0.12, -0.26, 0.01, 0.09, -0.34, -17.65, -0.3..."
128772,2019,2019122912,297,2938,29,2019-12-29T18:12:07.900,45719.0,"[[31.89, 45.37, 0.53, 5.22, 1.26, 0.0, 0.0, 31...","[[31.94, 45.02, 0.53, 4.83, 0.87, 71.97, 67.83...","[[32.43, 45.19, 0.52, 4.97, 1.08, 68.27, 70.45...",[-0.49],"[[-0.49, -0.17, 0.01, -0.14, -0.21, 3.7, -2.62...","[[-0.45, -0.13, 0.05, -0.16, -0.36, 0.0, -2.94..."


In [115]:
# all_ball_carriers = np.unique(test_data.playerId)

# len(all_ball_carriers)


In [182]:
'''
Returns float
'''
def calc_yards_ran(calc_preds_output):
    try:
        play_states_cpu = calc_preds_output.detach().cpu()
    except:
        play_states_cpu = calc_preds_output
    start_x = play_states_cpu[0][0,2]
    end_x = play_states_cpu[-2][0,2]
    yards_ran = end_x - start_x

    return round(float(yards_ran), 3)

'''
Adds to goe dictionary for play
'''
def calc_for_play(test_index, goe_dictionary):

    play_to_viz = get_returned_play(test_index, test_data, returned_plays)
    returnerId = float(play_to_viz['returnerId'])
    first_state = get_tensor_state(play_to_viz, device)
    home_away = get_home_away(play_to_viz, device)

    # (num_seconds, 23, 8)
    states_from_true, states_from_predicted, all_true_states, total_loss = calc_preds(play_to_viz)

    true_yards_ran = calc_yards_ran(all_true_states)
    expected_yards_ran = calc_yards_ran(states_from_predicted)
    goe = round(expected_yards_ran - true_yards_ran, 3)

    # add to dictionary
    if returnerId in goe_dictionary:
        goe_dictionary[returnerId].append(goe)
    else:
        goe_dictionary[returnerId] = [goe]
        
    return goe_dictionary

In [193]:
from collections import defaultdict

'''
GOE (gained over expected) = True yards ran - Expected yards ran

Create a dictionary. Keys are ball carrier, value is list of GOE
'''

goe_dictionary = defaultdict(list)

In [196]:
num_test_points = len(test_data)

start_time = time.time()

for i in range(25,385):
    try:
        goe_dictionary = calc_for_play(i, goe_dictionary)
    except Exception as e:
        print(f"error with {e}")
        continue
        
    if i % 500 == 0:
        print(f"Finished with {i} out of {num_test_points}")
        
end_time = time.time()

total_time = round(end_time - start_time, 3)

print(F"total time: {total_time}  seconds")

/tmp/ipykernel_2838/737660972.py:2: RuntimeWarning: invalid value encountered in subtract
  dist = np.linalg.norm(player_pos.values - x.values)


total time: 4599.686  seconds


In [197]:
goe_dictionary

defaultdict(list,
            {45599.0: [0.119, -4.213],
             40130.0: [14.875, 0.411, -0.868, -19.181, -10.106],
             44837.0: [-0.274, -0.595, -423216.874, -16.896, 0.815, 2.063],
             39975.0: [9.715, -13.03, -8.887, 0.724, -0.124, nan, -0.124],
             45719.0: [-3.413, -1.933],
             52595.0: [-20.816, -2.703, 4.654],
             43663.0: [-2.083, 1.498, -7.221, 7.245],
             36473.0: [-1.495],
             39212.0: [-6.537, -5.376, -4.189],
             43475.0: [-6.849,
              -2.331,
              -14.82,
              nan,
              -4.684,
              -8037898125382.59,
              -9.986,
              -1.2772755996934149e+17,
              -6.202,
              8.949],
             47605.0: [-20.479, -8.62, 2.172],
             47932.0: [5.842, -5.129, -9.131, -5.129],
             47839.0: [-5.287, -0.74, -13.696, -226510225.9, -5.542, -0.683],
             52737.0: [-0.692],
             43406.0: [4.611, 6.118, -7

In [192]:
print(len(test_data))

28808


In [199]:
with open('goe_dict_first_385.pkl', 'wb') as file:
    pickle.dump(goe_dictionary, file)

In [217]:
pd.isna(goe_dictionary[43475.0][3])

True

In [225]:
'''
Create dictionary with final stats
'''
goe_dict_final_stat = dict()

for key, value in goe_dictionary.items():
    
    for index, item in enumerate(value):
        if pd.isna(item) or item > 100 or item < -100:
            value[index] = 0
            
    goe_dict_final_stat[key] = (round(sum(value), 3), len(value))
    

In [226]:
goe_dict_final_stat

{45599.0: (-4.094, 2),
 40130.0: (-14.869, 5),
 44837.0: (-14.887, 6),
 39975.0: (-11.726, 7),
 45719.0: (-5.346, 2),
 52595.0: (-18.865, 3),
 43663.0: (-0.561, 4),
 36473.0: (-1.495, 1),
 39212.0: (-16.102, 3),
 43475.0: (-35.923, 10),
 47605.0: (-26.927, 3),
 47932.0: (-13.547, 4),
 47839.0: (-25.948, 6),
 52737.0: (-0.692, 1),
 43406.0: (4.874, 6),
 42794.0: (-19.811, 6),
 42051.0: (-22.102, 6),
 35527.0: (-115.992, 17),
 45020.0: (-33.895, 5),
 46270.0: (-16.24, 4),
 37251.0: (-0.804, 6),
 43948.0: (-14.286, 4),
 48415.0: (-80.679, 4),
 52559.0: (1.306, 2),
 46200.0: (18.971, 1),
 46506.0: (-16.393, 8),
 41997.0: (-1.385, 1),
 44074.0: (-3.172, 5),
 42482.0: (-11.373, 1),
 44029.0: (-4.993, 3),
 41353.0: (-9.634, 8),
 39985.0: (3.64, 1),
 46116.0: (9.241, 4),
 52619.0: (-1.655, 5),
 29830.0: (-13.556, 1),
 52425.0: (-26.161, 2),
 47896.0: (5.391, 1),
 45102.0: (-1.759, 1),
 44964.0: (-6.646, 3),
 43349.0: (-25.122, 4),
 46256.0: (-28.366, 10),
 48988.0: (-4.652, 1),
 46329.0: (10.2

In [295]:
# load in player data
player_data = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/players.csv")

display(player_data.head())

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen


In [310]:
player_data.query("nflId == 37267 or nflId == 44837")

,nflId,height,weight,birthDate,collegeName,Position,displayName
573,44837,5-11,213,1995-10-04,Michigan,SS,Jabrill Peppers
2265,37267,72,208,1987-10-18,Miami (Fla.),P,Matt Bosher


In [296]:
goe_df = pd.DataFrame(goe_dict_final_stat.values(), columns=['goe', 'num'])

goe_df.loc[:,'goe/play'] = round(goe_df.loc[:,'goe'] / goe_df.loc[:,'num'], 3)
goe_df.loc[:,'nflId'] = goe_dict_final_stat.keys()
goe_df.loc[:,'nflId'] = goe_df.loc[:,'nflId'].astype('int64')


In [297]:
goe_df

,goe,num,goe/play,nflId
0,-4.094,2,-2.047,45599
1,-14.869,5,-2.974,40130
2,-14.887,6,-2.481,44837
3,-11.726,7,-1.675,39975
4,-5.346,2,-2.673,45719
...,...,...,...,...
140,1.728,1,1.728,41984
141,-6.292,1,-6.292,41325
142,0.789,1,0.789,46192
143,3.783,1,3.783,37200


In [298]:
goe_viz = pd.merge(goe_df, player_data, how='left', on='nflId').sort_values('goe/play', ascending=False)

In [299]:
goe_viz

,goe,num,goe/play,nflId,height,weight,birthDate,collegeName,Position,displayName
24,18.971,1,18.971,46200,6-2,230,1995-12-22,Arizona State,RB,Kalen Ballage
46,14.262,1,14.262,46320,6-0,200,1996-04-22,Northwestern,RB,Justin Jackson
79,25.187,2,12.594,43396,6-1,200,1993-06-16,Cincinnati,WR,Chris Moore
100,11.982,1,11.982,52584,6-0,200,1997-06-10,Miami,WR,K.J. Osborn
139,11.209,1,11.209,46176,6-4,252,1996-02-23,Miami,TE,Christopher Herndon
...,...,...,...,...,...,...,...,...,...,...
127,-16.020,1,-16.020,44941,6-3,205,1996-03-21,Tennessee,WR,Josh Malone
61,-19.067,1,-19.067,52622,6-0,199,1998-08-04,Florida,WR,Freddie Swain
22,-80.679,4,-20.170,48415,5-6,170,1997-12-04,Assumption,WR,Deonte Harris
105,-52.226,2,-26.113,47843,6-0,206,1997-05-31,Delaware,FS,Nasir Adderley


In [300]:
display(goe_viz.query("num > 1").reset_index().iloc[0:15,:])

,index,goe,num,goe/play,nflId,height,weight,birthDate,collegeName,Position,displayName
0,79,25.187,2,12.594,43396,6-1,200,1993-06-16,Cincinnati,WR,Chris Moore
1,66,31.432,4,7.858,44923,6-0,176,1993-11-21,Oklahoma,WR,Dede Westbrook
2,104,12.212,2,6.106,41418,6-0,190,07/19/1992,Notre Dame,WR,T.J. Jones
3,65,11.978,2,5.989,48304,5-11,188,1995-11-01,North Dakota State,WR,Darrius Shepherd
4,125,10.583,2,5.292,43362,6-1,211,1994-01-26,Alabama,RB,Kenyan Drake
5,72,20.270,5,4.054,45150,5-7,153,1994-02-01,Akron,WR,JoJo Natson
6,120,7.959,2,3.980,46509,6-1,210,1995-08-08,Fresno State,WR,Da'Mari Scott
7,54,31.782,9,3.531,46265,6-0,190,1996-07-20,Central Arkansas,CB,Tremon Smith
8,51,6.206,2,3.103,42450,5-10,192,1991-12-18,East Carolina,WR,Justin Hardy
9,55,5.158,2,2.579,52433,6-1,206,1998-03-17,Arizona State,WR,Brandon Aiyuk


In [301]:
filtered_df = goe_viz.query("num > 1").reset_index()

filtered_df.drop(columns=['index'], inplace=True)


In [292]:
#filtered_df = filtered_df.style.set_properties(**{'background-color':'yellow'}, subset=['goe/play'])

In [303]:
with open('goe_best_df.pkl', 'wb') as file:
    pickle.dump(filtered_df, file)

In [304]:
worse_df = filtered_df.sort_values('goe/play', ascending=True)

In [307]:
with open('goe_worst_df.pkl', 'wb') as file:
    pickle.dump(worse_df, file)